In [65]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio'] + sys.path
import os 
import numpy as np 
import pandas as pd
from tqdm import tqdm 
from functools import lru_cache
from recstudio.data.advance_dataset import KDDCUPSessionDataset
import pickle
import copy 

In [55]:
valid_session_candidates_path = '/root/autodl-tmp/huangxu/valid_session_candidates_450.parquet'

In [62]:
@lru_cache(maxsize=1)
def read_valid_session_candidates():
    return pd.read_parquet(valid_session_candidates_path)

In [63]:
valid_session_candidates = read_valid_session_candidates()
valid_session_candidates.head(2)

,locale,candidates
0,UK,"[B06XG1LZ6Z, B06XGD9VLV, B076PN1SKG, B01MYUDYP..."
1,JP,"[B09LCPT9DQ, B09MRYK5CV, B0BB5VQ1L8, B084T9C6W..."


In [17]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [34]:
@lru_cache(maxsize=1)
def load_datasets_from_cache(data_dir):
    print(f"Load dataset from cache {data_dir}.")
    cache_datasets = _load_cache(data_dir)
    datasets = []
    for i in range(len(cache_datasets)):
        datasets.append(KDDCUPSessionDataset(None, data_dir, None, True))
        for k in cache_datasets[i].__dict__:
            attr = getattr(cache_datasets[i], k)
            setattr(datasets[i], k, attr)
    return datasets 

In [35]:
dataset = load_datasets_from_cache(
    '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/f30c8c1234f63ba5c60a8b7814777f60'
)[0]

Load dataset from cache /root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/f30c8c1234f63ba5c60a8b7814777f60.


In [36]:
item_token2id = dataset.field2token2idx['product_id']
locale_index = {
    'DE_index' : dataset.item_feat.get_col('DE_index'),
    'JP_index' : dataset.item_feat.get_col('JP_index'),
    'UK_index' : dataset.item_feat.get_col('UK_index'),
    'ES_index' : dataset.item_feat.get_col('ES_index'),
    'FR_index' : dataset.item_feat.get_col('FR_index'),
    'IT_index' : dataset.item_feat.get_col('IT_index'),
}

In [59]:
def token2pos(candidates_list, locale_name):
    cur_locale_index = locale_index[locale_name + '_index']
    candidates_id_list = [item_token2id[can] for can in candidates_list]
    candidates_pos_list = cur_locale_index[candidates_id_list]
    return np.array(candidates_pos_list)

In [67]:
valid_session_candidates_pos = copy.deepcopy(valid_session_candidates)
for i in tqdm(range(len(valid_session_candidates_pos))):
    valid_session_candidates_pos.at[i, 'candidates'] = \
        token2pos(valid_session_candidates_pos.iloc[i]['candidates'], valid_session_candidates_pos.iloc[i]['locale'])

  0%|          | 78/361581 [00:00<52:03, 115.74it/s] 


KeyboardInterrupt: 